In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [2]:
def test_f(X):
    return np.sin(X)+ np.sin(10*X/3)+1


X = np.random.random((10,1))*100
Y = test_f(X)

print(X,Y)

[[97.0200688 ]
 [69.52875623]
 [48.18704536]
 [63.38946701]
 [65.8882988 ]
 [29.12812929]
 [ 7.77809364]
 [78.39163273]
 [41.86292971]
 [29.96000363]] [[ 1.54374658]
 [ 0.74614551]
 [-0.265392  ]
 [ 0.80383176]
 [ 0.80501626]
 [ 0.53754161]
 [ 2.71045136]
 [ 0.62218487]
 [ 1.11368932]
 [-0.60990655]]


In [5]:
class continuous_kg():
    
    def __init__(self, model, space, optimizer):
        self.model = model
        self.space = space
        self.optimizer = optimizer
    def _compute_acq(self, X ):
        """
        Computes the aquisition function
        
        :param X: set of points at which the acquisition function is evaluated. Should be a 2d array.
        """
        # print("_compute_acq")
        self.update_current_best()
        X =np.atleast_2d(X)

        marginal_acqX = self._unconstrained_marginal_acq(X)
        KG = np.reshape(marginal_acqX, (X.shape[0],1))

        return KG
    
    
    def _unconstrained_marginal_acq(self, X):

        varX_obj = self.model.posterior_variance(X, noise=True)

        acqX = np.zeros((X.shape[0], 1))

        for i in range(0, len(X)):
            x = np.atleast_2d(X[i])

            #For each x new precompute covariance matrices for
            self.model.partial_precomputation_for_covariance(x)
            self.model.partial_precomputation_for_covariance_gradient(x)

            aux_obj = np.reciprocal(varX_obj[:, i])
            #Create discretisation for discrete KG.
            if self.fixed_discretisation is False:
                if self.optimise_discretisation:
                    self.X_Discretisation = self._unconstrained_discretisation_X(index=i, X=X, aux_obj =aux_obj )
                    # print("updated discretisation")
                    # print("quantiles", quantiles)
                    self.optimise_discretisation = False
            # print("X discretisation", self.X_Discretisation)

            kg_val = self.unconstrained_discrete_KG(Xd = self.X_Discretisation ,
                                      xnew = x,
                                      Zc=self.Z_cdKG,
                                      aux_obj =aux_obj)
            acqX[i,:] = kg_val

        return acqX.reshape(-1)

    def unconstrained_discrete_KG(self, Xd, xnew, Zc, aux_obj, grad=False, verbose=False):
        xnew = np.atleast_2d(xnew)
        # Xd = np.concatenate((Xd, self.fixed_discretisation_values))
        Xd = np.concatenate((Xd, xnew))
        Xd = np.concatenate((Xd, self.current_max_xopt))
        self.grad = grad
        out = []

        MM = self.model.predict(Xd)[0].reshape(-1)  # move up
        SS_Xd = np.array(self.model.posterior_covariance_between_points_partially_precomputed(Xd, xnew)[:, :, :]).reshape(-1)
        inv_sd = np.asarray(np.sqrt(aux_obj)).reshape(())

        SS = SS_Xd * inv_sd
        MM = MM.reshape(-1)
        SS = SS.reshape(-1)

        marginal_KG = []


        KG = self._unconstrained_parallel_KG(MM=MM,SS=SS,verbose=verbose)

        KG = np.clip(KG, 0, np.inf)
        marginal_KG.append(KG)
        out.append(marginal_KG)

        KG_value = np.mean(out)
        # gradKG_value = np.mean(gradout, axis=?)
        assert ~np.isnan(KG_value);
        "KG cant be nan"
        return KG_value#, gradKG_value

    def _unconstrained_parallel_KG(self, MM, SS, Xd=None,verbose=False):
        """
        Calculates the linear epigraph, i.e. the boundary of the set of points
        in 2D lying above a collection of straight lines y=a+bx.
        Parameters
        ----------
        a
            Vector of intercepts describing a set of straight lines
        b
            Vector of slopes describing a set of straight lines
        tol
            Minimum slope (in absolute value) different from zero
        Returns
        -------
        KGCB
            average hieght of the epigraph
        grad_a
            dKGCB/db, vector
        grad_b
            dKGCB/db, vector
        """
        a = MM #np.array(self.c_MM[:, index]).reshape(-1)
        b = SS #np.array(self.c_SS[:, index]).reshape(-1)

        assert len(b) > 0, "must provide slopes"
        assert len(a) == len(b), f"#intercepts != #slopes, {len(a)}, {len(b)}"
        # ensure 1D
        a = np.array(a).reshape(-1)
        b = np.array(b).reshape(-1)
        assert len(a) > 0, "must provide slopes"
        assert len(a) == len(b), f"#intercepts != #slopes, {len(a)}, {len(b)}"

        # ensure 1D
        a = np.atleast_1d(a.squeeze())
        b = np.atleast_1d(b.squeeze()) #- np.mean(b)
        max_a_index = np.argmax(a)
        maxa = np.max(a)
        n_elems = len(a)

        if np.all(np.abs(b) < 0.000000001):
            return np.array([0])#, np.zeros(a.shape), np.zeros(b.shape)

        # order by ascending b and descending a
        order = np.lexsort((-a, b))
        a = a[order]
        b = b[order]

        # exclude duplicated b (or super duper similar b)
        threshold = (b[-1] - b[0]) * 0.00001
        diff_b = b[1:] - b[:-1]
        keep = diff_b > threshold
        keep = np.concatenate([[True], keep])
        keep[np.argmax(a)] = True
        order = order[keep]
        a = a[keep]
        b = b[keep]

        # initialize
        idz = [0]
        i_last = 0
        x = [-np.inf]

        n_lines = len(a)
        # main loop TODO describe logic
        # TODO not pruning properly, e.g. a=[0,1,2], b=[-1,0,1]
        # returns x=[-inf, -1, -1, inf], shouldn't affect kgcb
        while i_last < n_lines - 1:
            i_mask = np.arange(i_last + 1, n_lines)
            x_mask = -(a[i_last] - a[i_mask]) / (b[i_last] - b[i_mask])

            best_pos = np.argmin(x_mask)
            idz.append(i_mask[best_pos])
            x.append(x_mask[best_pos])

            i_last = idz[-1]

        x.append(np.inf)

        x = np.array(x)
        idz = np.array(idz)

        # found the epigraph, now compute the expectation
        a = a[idz]
        b = b[idz]
        order = order[idz]

        pdf = norm.pdf(x)
        cdf = norm.cdf(x)

        KG = np.sum(a * (cdf[1:] - cdf[:-1]) + b * (pdf[:-1] - pdf[1:]))
        KG -= np.max(a)

        if KG - maxa <- 1e-5 :
            print("x",x)
            print("a",a )
            print("b",b)
            print("maxa",maxa)
            print("KG", KG)
            z_all = np.linspace(x[1], x[-2], 100)
            for j in range(len(a)):

                print(x[1], x[-2])
                if x[j] <-30:
                    z = np.linspace(-3, x[j+1], 3)
                elif x[j+1] > 30:
                    z = np.linspace(x[j], 3, 3)
                else:
                    z = np.linspace(x[j], x[j+1], 3)
                mu_star = a.reshape(-1)[j] + b.reshape(-1)[j]*z
                mu_star_all = a.reshape(-1)[j] + b.reshape(-1)[j]*z_all
                plt.plot(z_all , mu_star_all, color="grey")
                plt.plot(z, mu_star)
            plt.show()
            raise
        # if verbose:
        #     print("a_sorted ",a_sorted )
        #     print("b_sorted", b_sorted)
        #
        #     print("current max",np.max(a) )
        #     print("idz", idz)
        #     print("a", a)
        #     print("b", b)
        #     plt.scatter(Xd.reshape(-1), np.array(self.c_MM[:, index]).reshape(-1))
        #     plt.plot(np.linspace(0,5,2), np.repeat(np.max(a), 2), color="red")
        #     plt.show()
        #     # raise
        # if KG < -1e-5:
        #     print("KG cant be negative")
        #     print("np.sum(a * (cdf[1:] - cdf[:-1]) + b * (pdf[:-1] - pdf[1:]))",
        #           np.sum(a * (cdf[1:] - cdf[:-1]) + b * (pdf[:-1] - pdf[1:])))
        #     print("self.bases_value[index]", np.max(a))
        #     print("KG", KG)
        #     raise
        #
        # KG = np.clip(KG, 0, np.inf)

        if np.isnan(KG):
            print("KG", KG)
            print("self.bases_value[index]", max_a_index)
            print("a", a)
            print("b", b)
            raise

        return KG
    
    def update_current_best(self):
        n_observations = self.model.get_X_values().shape[0]
       
        print("updating current best..........")
        self.counter = n_observations
        self.current_max_xopt, self.current_max_value = self._compute_current_max()
        assert self.current_max_value.reshape(-1) is not np.inf; "error ocurred updating current best"
    
    def _compute_current_max(self):

        inner_opt_x, inner_opt_val = self.optimizer.optimize_inner_func(f=self.current_func,
                                                                        f_df=None,
                                                                        num_samples=5000)
        inner_opt_x = np.array(inner_opt_x).reshape(-1)
        inner_opt_x = np.atleast_2d(inner_opt_x)
        return inner_opt_x,-inner_opt_val
    
    def current_func(self,X_inner):
        X_inner = np.atleast_2d(X_inner)
        mu = self.model.posterior_mean(X_inner)[0]
        mu = np.array(mu).reshape(-1)
        return -(mu ).reshape(-1)

In [70]:
import os
import sys

sys.path.append(os.getcwd())

import core.acquisition.GPyOpt
from core.acquisition.Hybrid_continuous_KG_v2 import KG
from core.acquisition.multi_outputGP import multi_outputGP
import core.acquisition.GPy as GPy
import core.acquisition.GPy as GPy

space =  GPyOpt.Design_space(space =[{'name': 'var_1', 'type': 'continuous', 'domain': (0,100)},
                                     {'name': 'var_2', 'type': 'continuous', 'domain': (0,100)}])#GPyOpt.Design_space(space =[{'name': 'var_1', 'type': 'continuous', 'domain': (0,100)}])#


n_f = 1
model = multi_outputGP(output_dim=n_f, noise_var=[1e-3] * n_f, exact_feval=[True] * n_f)  # , normalizer=True)

model.trainModel(X, [Y])
acq_opt = GPyOpt.optimization.AcquisitionOptimizer(optimizer='lbfgs', 
                                                   space=space, 
                                                   model=model,
                                                   model_c=None)
acquisition = continuous_kg(model=model, 
                             space=space,
                            optimizer=acq_opt)

reconstraining parameters GP_regression.rbf
reconstraining parameters GP_regression.Gaussian_noise.variance


create model
noise_var 0.001
Optimization restart 1/1, f = 24.047509205227165
self.model 
Name : GP regression
Objective : 24.047509205227165
Number of Parameters : 3
Number of Optimization Parameters : 2
Updates : True
Parameters:
  GP_regression.           |               value  |  constraints  |      priors    
  rbf.variance             |   1.412667169866644  |      +ve      |    Ga(1, 0.5)  
  rbf.lengthscale          |  0.8014207602015717  |      +ve      |    Ga(1, 0.5)  
  Gaussian_noise.variance  |               0.001  |   +ve fixed   |  Ga(0.01, 0.01)


In [71]:
X_plot = np.linspace(0,100,40)

acq_vals = acquisition._compute_acq(X_plot)

plt.plot(X_plot, acq_vals)
plt.show()

updating current best..........


AttributeError: 'GPRegression' object has no attribute 'posterior_mean'

In [57]:
import inspect
inspect.getmodule(GPyOpt.models.GPModel).__file__

'/home/juan/Documents/Github_repos/Constrained-KG/core/acquisition/GPyOpt/models/gpmodel.py'

In [69]:

inspect.getmodule(GPy.models.GPRegression).__file__

'/home/juan/Documents/Github_repos/Constrained-KG/core/acquisition/GPy/models/gp_regression.py'